In [1]:
import numpy as np
import cv2 as cv
import time
import numpy as np
from torchvision.utils import draw_bounding_boxes 
import torchvision


In [2]:
from ultralytics import YOLO
from deepface import DeepFace

In [3]:
from deepface.extendedmodels import Emotion

In [4]:
senti_model = Emotion.loadModel()

In [60]:
results = model("videoplayback.mp4", stream=True)

Ultralytics YOLOv8.0.28 🚀 Python-3.9.16 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 3902MiB)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs


In [26]:
labels = {0: "angry", 1: "disgust", 2:"fear", 3:"happy", 4:"sad", 5:"surprise", 6:"neutral"}


In [31]:

def draw(image, bounding_boxes, frame_width, frame_height, id=1):
    dh, dw, _ = image.shape
    for x, y,w,h in bounding_boxes:

        # Draw a rectangle on the image
        x = int(x) 
        y = int(y) 
        w = int(w) 
        h = int(h)

        l = int((x - w / 2))
        r = int((x + w / 2))
        t = int((y - h / 2))
        b = int((y + h / 2))

        image = cv2.rectangle(image, (l, t), (r, b), (0, 0, 255), 2)        
        # img = cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)  # (0, 255, 0) is the color in BGR format
        cropped_image = image[t:b, l:r]
        cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
        cropped_image = cv.resize(cropped_image, (48,48))
        result = senti_model.predict(np.expand_dims(cropped_image, axis=0), verbose=False)
        emotion = np.argmax(result)
        image = cv2.putText(image, labels[emotion], (l, t - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


    return image 

In [32]:
results = model("videoplayback.mp4", stream=True)import cv2
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

# Open the video file
video_path = "/home/sawaiz/Documents/Lab/In Progress/Video face detection/notebooks/videoplayback.mp4"
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output1.avi',cv2.VideoWriter_fourcc('M','J','P','G'), fps, (frame_width,frame_height))

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model(frame, verbose=False, classes=[0], conf= 0.4)

        # Visualize the results on the frame
        annotated_frame = draw(frame, results[0].boxes.xywh, frame_width, frame_height, id=1)
        
        out.write(annotated_frame)

        # Display the annotated frame
        # cv2.imshow("YOLOv8 Tracking", annotated_frame)
    else:
        break

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release() # Release the video file
cv2.destroyAllWindows()


In [ ]:
#42 seconds for a two minute video
#54 second for a resized. So no need to resize
#just cropping takes 43 seconds
#Takes really long with sentiment analyzer of deepface ofc



In [6]:
def resize_image(srcimg,input_height, input_width, keep_ratio=True):
    if keep_ratio and srcimg.shape[0] != srcimg.shape[1]:
        hw_scale = srcimg.shape[0] / srcimg.shape[1]
        if hw_scale > 1:
            newh, neww = input_height, int(input_width / hw_scale)
            img = cv2.resize(srcimg, (neww, newh), interpolation=cv2.INTER_AREA)
        else:
            newh, neww = int(input_height * hw_scale), input_width
            img = cv2.resize(srcimg, (neww, newh), interpolation=cv2.INTER_AREA)
    else:
        img = cv2.resize(srcimg, (input_width, input_height), interpolation=cv2.INTER_AREA)
    return img 

In [4]:
video_file_path = '/home/sawaiz/Documents/Lab/In Progress/Video face detection/experimentation_videos/videoplayback.mp4'  # Replace with the path to your video file
cap = cv.VideoCapture(video_file_path)
fps = cap.get(cv.CAP_PROP_FPS)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fourcc = cv.VideoWriter_fourcc(*'XVID')
out = cv.VideoWriter('output1.avi',cv.VideoWriter_fourcc('M','J','P','G'), fps/2, (frame_width,frame_height))
if not cap.isOpened():
    print("Error: Could not open video file.")

start_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    frame = resize_image(frame, frame_height, frame_width)


    # cv.imshow("Img", frame)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
out.release() # Release the video file
# cv.destroyAllWindows()

end_time = time.time()
elapsed_time = end_time - start_time
print(f"For the model, Elapsed time: {elapsed_time} seconds")


For the model, Elapsed time: 20.2766056060791 seconds
